In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from glob import glob
from operator import itemgetter
from persiantools import characters
from persiantools.jdatetime import JalaliDate
import persiantools
import datetime
import re
from re import search
import itertools
from statistics import mode

In [ ]:
def excel_to_csv(path,dirr):
    extention='xlsx'
    os.chdir(path)
    for file in os.listdir(path):
        file_xlsx= pd.read_ecxel(file)
        file_xlsx.to_csv(dirr+"\\"+file.split('.')[0]+".csv",index=False)

In [ ]:
def csv_to_xlsx(path,dirr):
    for file in os.listdir(path):
        df=pd.read_csv(path+'\\'+file)
        df.to_excel(dirr+'\\'+file.split('.')[0]+'xlsx',index=False)

In [ ]:
def read_merge_output(path,dirr=None):
    input_1=input('select the type of file: (csv/xlsx) ')
    all_df=[]
    if input_1=='xlsx':
        input_2=input('insert the number of columns: ')
        input_2= int(input_2)
        for file in os.listdir(path):
            df=pd.read_xlsx(path+'\\'+file)
            file_name= file.split('.')[0]
            column_list= df.columns.tolist()
            len_df= len(column_list)
            if len_df== input_2:
                all_df.append(df)
            else:
                print(f'the number of columns of file {file_name} is {len_df}')
                input_3= input('remove column or add column: (add/remove) ')
                if input_3 == 'add':
                    input_4= input('name of column: ')
                    df[input_4]=None
                    all_df.append(df)
                if input_3=='remove':
                    input_5= input('name of removed column: ')
                    df.drop(columns=input_5)
                    all_df.append(df)
    if input_1=='csv':
        input_2=input('insert the number of columns: ')
        input_2= int(input_2)
        for file in os.listdir(path):
            df=pd.read_csv(path+'\\'+file)
            file_name= file.split('.')[0]
            column_list= df.columns.tolist()
            len_df= len(column_list)
            if len_df== input_2:
                all_df.append(df)
            else:
                print(f'the number of columns of file {file_name} is {len_df}')
                input_3= input('remove column or add column: (add/remove) ')
                if input_3 == 'add':
                    input_4= input('name of column: ')
                    df[input_4]=None
                    all_df.append(df)
                if input_3=='remove':
                    input_5= input('name of removed column: ')
                    df.drop(columns=input_5)
                    all_df.append(df)
    input_6=input('do you want output: (y/n) ')
    if input_6=='y':
        df_raw= pd.concat(all_df,ignore_index=True).reset_index(drop=True)
        input_7=input('file type: (xlsx\csv) ')
        if input_7== 'xlsx':
            input_8= ('file name: ')
            df_raw.to_excel(dirr+'\\'+input_8+'.xlsx',index=False)
        if input_7== 'csv':
            input_8= ('file name: ')
            df_raw.to_csv(dirr+'\\'+input_8+'.csv',index=False)
    if input_6=='n':
        df_raw= pd.concat(all_df,ignore_index=True).reset_index(drop=True)
    return df_raw

In [ ]:
def change_column_name(df):
    columns= df.columns.tolist()
    print(f'column name is: {columns}')
    input_1= input('do you want to change header: (n\y)')
    if input_1=='y':
        for i in columns:
            input_2= input(f'insert new name for {i}: ')
            df.rename(columns={i:input_2},inplace=True)

In [ ]:
def column_clustering(df):
    columns= df.columns.tolist()
    print(f'column name is: {columns}')
    string_columns=[]
    code_columns=[]
    price_columns=[]
    input_1=None
    while input_1!='finish':
        input_1= input('choose type of clusters: (string\price\code) ')
        if input_1=='string':
            input_2=None
            while input_2!='finish':
                input_2=input('choose string column')
                string_columns.append(input_2)
            if 'finish' in string_columns:
                string_columns.remove('finish')
        if input_1=='code':
            input_2=None
            while input_2!='finish':
                input_2=input('choose code column')
                code_columns.append(input_2)
            if 'finish' in code_columns:
                code_columns.remove('finish')
        if input_1=='price':
            input_2=None
            while input_2!='finish':
                input_2=input('choose price column')
                price_columns.append(input_2)
            if 'finish' in price_columns:
                price_columns.remove('finish')

    return string_columns,code_columns,price_columns
            

In [ ]:
def code_clean(df, code_columns):
    for i in code_columns:
        try:
            df[i]= df[i].replace(['nan',np.nan,'NaN'],1000000000001234)
            df[i]=df[i].replace(r'-','',regex=True)
            df[i]=df[i].apply(np.int64)
            df[i]= df[i].astype(str)
            df[i]= df[i].replace(r'1000000000001234',np.nan, regex=True)
        except:
            print(f'error is in {i}')
    return df

In [ ]:
def price_clean (df, price_columns):
    input_1= input('type of price: (float/int)')
    if input_1=='int':
        for i in price_columns:
            try:
                df[i]= df[i].apply(lambda x: x.astype(str))
                df[i]=df[i].apply(lambda x: x(np.int64))
            except:
                print(f'error is in {i}')
    if input_1=='float':
        for i in price_columns:
            try:
                df[i]= df[i].apply(lambda x: x.astype(float))
            except:
                print(f'error is in {i}')
    return df


In [ ]:
def string_clean(df,string_columns):
    for i in string_columns:
        try:
            df[i]= df[i].apply(lambda x: x.astype(str))
            df[i]=df[i].str.lower()
            df[i]=df[i].str.strip()
            df[i]=df[i].replace('  ',' ')
            df[i]=df[i].replace('(','-')
            df[i]=df[i].replace(')','')
            df[i]=df[i].replace('_',' ')
            df[i]=df[i].apply(lambda x: characters.ar_to_fa(x))
            df[i]=df[i].replace(regex=r'\u200c',value=' ')
            df[i]=df[i].str.strip(' ')
            df[i]=df[i].replace(regex=['مجتمع','سهامی خاص','شرکت','سهامی عام'],value='')
        except:
            print(f'error is in {i}')
    return df

In [ ]:
def P_E (df):
    def unique_value(df,column):
        x= df[column].drop_duplicates()
        return x
    columns= df.columns.tolist()
    print(f'list of columns: {columns}')
    columns_list=[]
    input_1=None
    while input_1 != 'finish':
        input_1= input('select column: ')
        columns_list.append(input_1)
    if 'finish' in columns_list:
        columns_list.remove('finish')
    for j in columns_list:
        u = unique_value(df,[j])
        input_2= input(f'{j} in your dataframe has {len(u)} unique value; do you want to replace: (N/Y)')
        if input_2=='n':
            continue
        else:
            dict_replace={}
            c=1
            for i in u.iloc[:,0]:
                input_3= input(f'{c} what is replace of {i}')
                dict_replace[i]= input_3
                if input_3 =='break':
                    break
                else:
                    c+=1
        input_4= input('do you want creat new column?')
        if input_4 == 'break':
            break
        else:
            if input_4=='y':
                name=''
                input_5= input('name of new column: ')
                df[f'{input_5}']= df[j].replace(dict_replace)
            else:
                df[j]= df[j].replace(dict_replace)


In [ ]:
def wprds_clean(df,columns_list:list):
    for j in columns_list:
        v= df[j].value_counts()
        input_1= input(f'{j} has: {v}/n do wou want to clean: (n/y)')
        if input_1=='n':
            continue
        else:
            list_replace=[]
            input_2=None
            while input_2 != 'finish':
                input_2= input ('what is the words to remove: ')
                list_replace.append(input_2)
            if 'finish' in list_replace:
                list_replace.remove('finish')
            input_3= input('do you want to creat new column: ')
            if input_3 =='break':
                break
            else:
                if input_3=='y':
                    name=''
                    input_4= input('new column name: ')
                    df[f'{input_4}']= df[j].replace(regex=list_replace,value='')
                else:
                    df[j]= df[j].replace(regex=list_replace,value='')

In [ ]:
def remove_extra_words (pd_series):
    pd_series= pd_series.astype(str)
    pd_series= pd_series.replace(r'سهامی عام','',regex=True)
    pd_series= pd_series.replace(r'شرکت','',regex=True)
    pd_series= pd_series.replace(r'مجتمع','',regex=True)
    pd_series= pd_series.replace(r'سهامی خاص','',regex=True)
    pd_series= pd_series.str.strip()
    pd_series= pd_series.str.strip(' ')
    return pd_series

In [ ]:
def remove_extra_space (pd_series):
    pd_series=pd_series.astype(str)
    pd_series=pd_series.str.strip(' ')
    return pd_series

In [ ]:
def remove_space(pd_series):
    pd_series= pd_series.astype(str)
    pd_series.replace(r' ','',regex=True)
    return pd_series

In [ ]:
def date_timestamp(pd_series):
    pd_series=pd_series.astype(str)
    date = pd_series.apply (lambda x: str(x).split(' ')[0])
    return date

In [ ]:
def date_corrector(pd_series):
    x= pd_series.astype(str)
    year= x.apply(lambda x: str(x)[0:4])
    month= x.apply(lambda x: str(x)[4:6])
    day= x.apply(lambda x: str(x)[6:])
    date = year +'-'+month+'-'+day
    pd_series= date
    pd_series= pd_series.replace(regex='<NA>--', value= np.nan)
    return pd_series

In [ ]:
def finding_value_str(df):
    y = df.columns.tolist()
    print('Note: it takes case sensitive values.')
    keyword= input('keyword for search')
    dict_name={}
    for k in y:
        try:
            l= df[df[k].str.contains(keyword)]
            dict_name[k]= l.index.tolist()
        except:
            continue
    print(dict_name)

In [ ]:
def view(df):
    css = """<style>
    table {border-collapse: collapse; border: 3px solid #eee; }
    table tr th: first-child {background-color: #eee; color: #000; }
    table thead th {background-color: #eee; color:#000 }
    tr, th, td {border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px: font-family: monospace; font-size: 10px }<style>
    """
    s= '<script type = "text/Javascript">'
    s+= 'var win = window.open("", "Title","toolbar=no", location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400+", left="+(screen.width-840"));'
    s+= (
        "win.document.body.innerHTML = '"
        + (df.to_html())+ css).replace("\n","\\")
        + "';"
    )
    s+= "</script>"
    return HTML(s+css)


Translate with regex

In [ ]:
def regex_translate(list_dict,pd_series='nan',string_columns='nan',df=np.nan,first_path='nan',second_path='nan'):
    def string_clean(df,string_columns):
        for i in string_columns:
            try:
                df[i]= df[i].apply(lambda x: x.astype(str))
                df[i]=df[i].str.lower()
                df[i]=df[i].str.strip()
                df[i]=df[i].replace('  ',' ')
                df[i]=df[i].replace('(','-')
                df[i]=df[i].replace(')','')
                df[i]=df[i].replace('_',' ')
                df[i]=df[i].apply(lambda x: characters.ar_to_fa(x))
                df[i]=df[i].replace(regex=r'\u200c',value=' ')
                df[i]=df[i].str.strip(' ')
                df[i]=df[i].replace(regex=['مجتمع','سهامی خاص','شرکت','سهامی عام'],value='')
            except:
                print(f'error is in {i}')
        return df
    def column_clustering(df):
        columns= df.columns.tolist()
        print(f'column name is: {columns}')
        string_columns=[]
        code_columns=[]
        price_columns=[]
        input_1=None
        while input_1!='finish':
            input_1= input('choose type of clusters: (string\price\code) ')
            if input_1=='string':
                input_2=None
                while input_2!='finish':
                    input_2=input('choose string column')
                    string_columns.append(input_2)
                if 'finish' in string_columns:
                    string_columns.remove('finish')
            if input_1=='code':
                input_2=None
                while input_2!='finish':
                    input_2=input('choose code column')
                    code_columns.append(input_2)
                if 'finish' in code_columns:
                    code_columns.remove('finish')
            if input_1=='price':
                input_2=None
                while input_2!='finish':
                    input_2=input('choose price column')
                    price_columns.append(input_2)
                if 'finish' in price_columns:
                    price_columns.remove('finish')

        return string_columns,code_columns,price_columns
    def change_column_name(df):
        columns= df.columns.tolist()
        print(f'column name is: {columns}')
        input_1= input('do you want to change header: (n\y)')
        if input_1=='y':
            for i in columns:
                input_2= input(f'insert new name for {i}: ')
                df.rename(columns={i:input_2},inplace=True)
    input_1= input('do you have file: (n\y)')
    if input_1=='y':
        input_2= input('do you want to run cleaning function (n\y)')
        if input_2=='n':
            input_3= input('name of new column: ')
            for i in list_dict:
                check_elem=f'{i}'
                for j in pd_series:
                    sorce= f'{j}'
                    if re.search(r'\b'+check_elem+ r'\b',source):
                        checked= df.loc[pd_series==f'{j}'].index.tolist()
                        try:
                            df.loc[checked, input_3]=i
                        except:
                            continue
        if input_2=='y':
            columns= df.columns.tolist()
            print(f'there are columns: {columns}')
            input_4= input('select string columns: ')
            string_clean(df,input_4)
            input_5= input('name of new column: ')
            for i in list_dict:
                check_elem=f'{i}'
                for j in pd_series:
                    sorce= f'{j}'
                    if re.search(r'\b'+check_elem+ r'\b',source):
                        checked= df.loc[pd_series==f'{j}'].index.tolist()
                        try:
                            df.loc[checked, input_5]=i
                        except:
                            continue
    if input_1=='n':
        input_2= input('choose file format: (xlsx\csv)')
        if input_2=='xlsx':
            input_3= input('select string columns: ')
            input_4= input('new column name: ')
            input_5= input('which column to check: ')
            for file in os.listdir(first_path):
                df= pd.read_excel(first_path+'\\'+file)
                df[input_3]= df[input_3].fillna('نامشخص')
                string_clean(df, input_3)
                for i in list_dict:
                    check_elem=f'{i}'
                    for j in input_5:
                        sorce= f'{j}'
                        if re.search(r'\b'+check_elem+ r'\b',source):
                            checked= df.loc[input_5==f'{j}'].index.tolist()
                            try:
                                df.loc[checked, input_4]=i
                            except:
                                continue
                df.to_csv(second_path+'\\'+file.split('.')[0]+'.csv',index=False)
        if input_2=='csv':
            input_3= input('select string columns: ')
            input_4= input('new column name: ')
            input_5= input('which column to check: ')
            for file in os.listdir(first_path):
                df= pd.read_csv(first_path+'\\'+file)
                df[input_3]= df[input_3].fillna('نامشخص')
                string_clean(df, input_3)
                for i in list_dict:
                    check_elem=f'{i}'
                    for j in input_5:
                        source= f'{j}'
                        if re.search(r'\b'+check_elem+ r'\b',source):
                            checked= df.loc[input_5==f'{j}'].index.tolist()
                            try:
                                df.loc[checked, input_4]=i
                            except:
                                continue
                df.to_csv(second_path+'\\'+file.split('.')[0]+'.csv',index=False)

In [ ]:
def regex_translate_tuple (list_dict,pd_seried='nan',string_columns='nan',df=np.nan,first_path='nan',second_path='nan'):
    def string_clean(df,string_columns):
        for i in string_columns:
            try:
                df[i]= df[i].apply(lambda x: x.astype(str))
                df[i]=df[i].str.lower()
                df[i]=df[i].str.strip()
                df[i]=df[i].replace('  ',' ')
                df[i]=df[i].replace('(','-')
                df[i]=df[i].replace(')','')
                df[i]=df[i].replace('_',' ')
                df[i]=df[i].apply(lambda x: characters.ar_to_fa(x))
                df[i]=df[i].replace(regex=r'\u200c',value=' ')
                df[i]=df[i].str.strip(' ')
                df[i]=df[i].replace(regex=['مجتمع','سهامی خاص','شرکت','سهامی عام'],value='')
            except:
                print(f'error is in {i}')
        return df
    def column_clustering(df):
            columns= df.columns.tolist()
            print(f'column name is: {columns}')
            string_columns=[]
            code_columns=[]
            price_columns=[]
            input_1=None
            while input_1!='finish':
                input_1= input('choose type of clusters: (string\price\code) ')
                if input_1=='string':
                    input_2=None
                    while input_2!='finish':
                        input_2=input('choose string column')
                        string_columns.append(input_2)
                    if 'finish' in string_columns:
                        string_columns.remove('finish')
                if input_1=='code':
                    input_2=None
                    while input_2!='finish':
                        input_2=input('choose code column')
                        code_columns.append(input_2)
                    if 'finish' in code_columns:
                        code_columns.remove('finish')
                if input_1=='price':
                    input_2=None
                    while input_2!='finish':
                        input_2=input('choose price column')
                        price_columns.append(input_2)
                    if 'finish' in price_columns:
                        price_columns.remove('finish')

            return string_columns,code_columns,price_columns
    def change_column_name(df):
        columns= df.columns.tolist()
        print(f'column name is: {columns}')
        input_1= input('do you want to change header: (n\y)')
        if input_1=='y':
            for i in columns:
                input_2= input(f'insert new name for {i}: ')
                df.rename(columns={i:input_2},inplace=True)
    input_1= input('do you have file: (n\y)')
    if input_1=='y':
        input_2= input('do you want to run cleaning function: (n\y)')
        if input_2=='n':
            input_3= input('name of new column: ')
            input_4= input('name of check column: ')
            df[input_3]='nan'
            tuple_list=[]
            for i in df[input_4]:
                list_cleaner=[]
                source=f'{i}'
                for j in list_dict:
                    check_elem=f'{j}'
                    if re.search(r'\b'+check_elem+ r'/b', source):
                        list_cleaner.append(j)
                tuple_cleanner= tuple(k for k in list_cleaner)
                tuple_list.append(tuple_cleanner)
            df[input_3]= tuple_list
        if input_2=='y':
            column_list= df.columns.tolist()
            print(f'there are the columns: {column_list}')
            input_3= input('select string column: ')
            string_clean(df, input_3)
            input_4= input('name of new column: ')
            input_5= input('the check column name: ')
            df[input_4]='nan'
            tuple_list=[]
            for i in df[input_5]:
                list_cleaner=[]
                source=f'{i}'
                for j in list_dict:
                    check_elem=f'{j}'
                    if re.search(r'\b'+check_elem+ r'/b', source):
                        list_cleaner.append(j)
                tuple_cleanner= tuple(k for k in list_cleaner)
                tuple_list.append(tuple_cleanner)
            df[input_4]= tuple_list
    if input_1=='n':
        input_2= input('choose format file: (xlsx\csv)')
        if input_2=='xlsx':
            input_3= input('select the string column: ')
            input_4= input('name of new column: ')
            input_5= input('the check column name: ')
            for file in os.listdir(first_path):
                df= pd.read_excel(first_path+'\\'+file)
                df[input_3]= df[input_3].fillna('نامشخص')
                string_clean(df, input_3)
                df[input_4]='nan'
                tuple_list=[]
                for i in df[input_5]:
                    list_cleaner=[]
                    source=f'{i}'
                    for j in list_dict:
                        check_elem=f'{j}'
                        if re.search(r'\b'+check_elem+ r'/b', source):
                            list_cleaner.append(j)
                    tuple_cleanner= tuple(k for k in list_cleaner)
                    tuple_list.append(tuple_cleanner)
                df[input_4]= tuple_list
                df.to_csv(second_path+'\\'+file.split('.')[0]+'.csv', index=False)
        if input_2=='csv':
            input_3= input('select the string column: ')
            input_4= input('name of new column: ')
            input_5= input('the check column name: ')
            for file in os.listdir(first_path):
                df= pd.read_csv(first_path+'\\'+file)
                df[input_3]= df[input_3].fillna('نامشخص')
                string_clean(df, input_3)
                df[input_4]='nan'
                tuple_list=[]
                for i in df[input_5]:
                    list_cleaner=[]
                    source=f'{i}'
                    for j in list_dict:
                        check_elem=f'{j}'
                        if re.search(r'\b'+check_elem+ r'/b', source):
                            list_cleaner.append(j)
                    tuple_cleanner= tuple(k for k in list_cleaner)
                    tuple_list.append(tuple_cleanner)
                df[input_4]= tuple_list
                df.to_csv(second_path+'\\'+file.split('.')[0]+'.csv', index=False)

In [ ]:
def regex_translate_tuple_new(list_dict,pd_seried='nan',string_columns='nan',df=np.nan,first_path='nan',second_path='nan'):
    def string_clean(df,string_columns):
        for i in string_columns:
            try:
                df[i]= df[i].apply(lambda x: x.astype(str))
                df[i]=df[i].str.lower()
                df[i]=df[i].str.strip()
                df[i]=df[i].replace('  ',' ')
                df[i]=df[i].replace('(','-')
                df[i]=df[i].replace(')','')
                df[i]=df[i].replace('_',' ')
                df[i]=df[i].apply(lambda x: characters.ar_to_fa(x))
                df[i]=df[i].replace(regex=r'\u200c',value=' ')
                df[i]=df[i].str.strip(' ')
                df[i]=df[i].replace(regex=['مجتمع','سهامی خاص','شرکت','سهامی عام'],value='')
            except:
                print(f'error is in {i}')
        return df
    def column_clustering(df):
            columns= df.columns.tolist()
            print(f'column name is: {columns}')
            string_columns=[]
            code_columns=[]
            price_columns=[]
            input_1=None
            while input_1!='finish':
                input_1= input('choose type of clusters: (string\price\code) ')
                if input_1=='string':
                    input_2=None
                    while input_2!='finish':
                        input_2=input('choose string column')
                        string_columns.append(input_2)
                    if 'finish' in string_columns:
                        string_columns.remove('finish')
                if input_1=='code':
                    input_2=None
                    while input_2!='finish':
                        input_2=input('choose code column')
                        code_columns.append(input_2)
                    if 'finish' in code_columns:
                        code_columns.remove('finish')
                if input_1=='price':
                    input_2=None
                    while input_2!='finish':
                        input_2=input('choose price column')
                        price_columns.append(input_2)
                    if 'finish' in price_columns:
                        price_columns.remove('finish')

            return string_columns,code_columns,price_columns
    def change_column_name(df):
        columns= df.columns.tolist()
        print(f'column name is: {columns}')
        input_1= input('do you want to change header: (n\y)')
        if input_1=='y':
            for i in columns:
                input_2= input(f'insert new name for {i}: ')
                df.rename(columns={i:input_2},inplace=True)
    input_1= input('do you have file: (n\y)')
    if input_1=='y':
        input_2= input('do you want to run cleaning function: (n\y)')
        if input_2=='n':
            input_3= input('name of new column: ')
            input_4= input('name of check column: ')
            df[input_3]=[()for i in df.index]
            for i in list_dict:
                check_elem=f'{j}'
                checked_index=[]
                for j in df[input_4]:
                    source=f'{j}'
                    list_cleaner=[]
                    if re.search(r'\b'+check_elem+ r'/b', source):
                        list_cleaner.append(j)
                        insert_list=[]
                        tuple_cleanner= tuple(k for k in list_cleaner)
                        item_index= df.loc[df[input_4]==source].index.tolist()
                        for x in item_index:
                            if x in checked_index:
                               item_index.remove(x)
                            else:
                                checked_index.append(x)
                                insert_list.append(x)
                        df.loc[insert_list, input_3].apply(lambda x: x+tuple_cleanner) 
        if input_2=='y':
            column_list= df.columns.tolist()
            print(f'there are the columns: {column_list}')
            input_3= input('select string column: ')
            string_clean(df, input_3)
            input_4= input('name of new column: ')
            input_5= input('the check column name: ')
            df[input_4]=[()for i in df.index]
            for i in list_dict:
                check_elem=f'{j}'
                checked_index=[]
                for j in df[input_5]:
                    source=f'{j}'
                    list_cleaner=[]
                    if re.search(r'\b'+check_elem+ r'/b', source):
                        list_cleaner.append(j)
                        insert_list=[]
                        tuple_cleanner= tuple(k for k in list_cleaner)
                        item_index= df.loc[df[input_5]==source].index.tolist()
                        for x in item_index:
                            if x in checked_index:
                               item_index.remove(x)
                            else:
                                checked_index.append(x)
                                insert_list.append(x)
                        df.loc[insert_list, input_4].apply(lambda x: x+tuple_cleanner) 
    if input_1=='n':
        input_2= input('choose format file: (xlsx\csv)')
        if input_2=='xlsx':
            input_3= input('select the string column: ')
            input_4= input('name of new column: ')
            input_5= input('the check column name: ')
            for file in os.listdir(first_path):
                df= pd.read_excel(first_path+'\\'+file)
                df[input_3]= df[input_3].fillna('نامشخص')
                string_clean(df, input_3)
            df[input_4]=[()for i in df.index]
            for i in list_dict:
                check_elem=f'{j}'
                checked_index=[]
                for j in df[input_5]:
                    source=f'{j}'
                    list_cleaner=[]
                    if re.search(r'\b'+check_elem+ r'/b', source):
                        list_cleaner.append(j)
                        insert_list=[]
                        tuple_cleanner= tuple(k for k in list_cleaner)
                        item_index= df.loc[df[input_5]==source].index.tolist()
                        for x in item_index:
                            if x in checked_index:
                               item_index.remove(x)
                            else:
                                checked_index.append(x)
                                insert_list.append(x)
                        df.loc[insert_list, input_4].apply(lambda x: x+tuple_cleanner)
                df.to_csv(second_path+'\\'+file.split('.')[0]+'.csv', index=False)
        if input_2=='csv':
            input_3= input('select the string column: ')
            input_4= input('name of new column: ')
            input_5= input('the check column name: ')
            for file in os.listdir(first_path):
                df= pd.read_csv(first_path+'\\'+file)
                df[input_3]= df[input_3].fillna('نامشخص')
                string_clean(df, input_3)
            df[input_4]=[()for i in df.index]
            for i in list_dict:
                check_elem=f'{j}'
                checked_index=[]
                for j in df[input_5]:
                    source=f'{j}'
                    list_cleaner=[]
                    if re.search(r'\b'+check_elem+ r'/b', source):
                        list_cleaner.append(j)
                        insert_list=[]
                        tuple_cleanner= tuple(k for k in list_cleaner)
                        item_index= df.loc[df[input_5]==source].index.tolist()
                        for x in item_index:
                            if x in checked_index:
                               item_index.remove(x)
                            else:
                                checked_index.append(x)
                                insert_list.append(x)
                        df.loc[insert_list, input_4].apply(lambda x: x+tuple_cleanner)
                df.to_csv(second_path+'\\'+file.split('.')[0]+'.csv', index=False)

Optimize

In [ ]:
def regex_translate_tuple_optimize(list_dict,pd_seried='nan',string_columns='nan',df=np.nan,first_path='nan',second_path='nan'):
    def string_clean(df,string_columns):
        for i in string_columns:
            try:
                df[i]= df[i].apply(lambda x: x.astype(str))
                df[i]=df[i].str.lower()
                df[i]=df[i].str.strip()
                df[i]=df[i].replace('  ',' ')
                df[i]=df[i].replace('(','-')
                df[i]=df[i].replace(')','')
                df[i]=df[i].replace('_',' ')
                df[i]=df[i].apply(lambda x: characters.ar_to_fa(x))
                df[i]=df[i].replace(regex=r'\u200c',value=' ')
                df[i]=df[i].str.strip(' ')
                df[i]=df[i].replace(regex=['مجتمع','سهامی خاص','شرکت','سهامی عام'],value='')
            except:
                print(f'error is in {i}')
        return df
    def column_clustering(df):
            columns= df.columns.tolist()
            print(f'column name is: {columns}')
            string_columns=[]
            code_columns=[]
            price_columns=[]
            input_1=None
            while input_1!='finish':
                input_1= input('choose type of clusters: (string\price\code) ')
                if input_1=='string':
                    input_2=None
                    while input_2!='finish':
                        input_2=input('choose string column')
                        string_columns.append(input_2)
                    if 'finish' in string_columns:
                        string_columns.remove('finish')
                if input_1=='code':
                    input_2=None
                    while input_2!='finish':
                        input_2=input('choose code column')
                        code_columns.append(input_2)
                    if 'finish' in code_columns:
                        code_columns.remove('finish')
                if input_1=='price':
                    input_2=None
                    while input_2!='finish':
                        input_2=input('choose price column')
                        price_columns.append(input_2)
                    if 'finish' in price_columns:
                        price_columns.remove('finish')

            return string_columns,code_columns,price_columns
    def change_column_name(df):
        columns= df.columns.tolist()
        print(f'column name is: {columns}')
        input_1= input('do you want to change header: (n\y)')
        if input_1=='y':
            for i in columns:
                input_2= input(f'insert new name for {i}: ')
                df.rename(columns={i:input_2},inplace=True)
    def condition(df, check, source):
        list_cleanner=[]
        if re.serach(r'\b'+ check +r'\b',source):
            list_cleanner.append(check)
            tuple_cleanner= tuple(k for k in list_cleanner)
            each_index= df.loc[df[input_6]==sorce].index.tolist()
            checked_index.extend([x for x in each_index if x not in check_index])
            insert_list.extend([x for x in each_index if x not in check_index])
            for x in elem_index:
                if x in checked_index:
                    elem_index.remove(x)
                else:
                    checked_index.append(x)
                    insert_index.append(x)
            df.loc[each_index,input_3]= df.loc[each_index, input_3].apply (lambda x: x+tuple_cleanner)
    input_1= input('do you have file: (n\y)')
    input_2= input('do you want cleannin function: (n\y)')
    input_3= input('name of new column: ')
    input_4= input('select string column: ')
    input_5= input('choose file format: (xlsx\csv)')
    input_6= input('which column to check: ')
    if input_1=='y':
        if input_2=='n':
            df[input_3]=[() for i in df.index]
            list_check= df[input_6].drop_duplicates().tolist()
            for i,j in itertools.product(list_dict, list_check):
                sorce= f'{j}'
                check=f'{i}'
                filter(condition(df,check,source), df[input_6])
            return df
        if input_2=='y':
            columns_list= df.columns.tolist()
            print(f'there are the columns: ')
            string_clean(df, input_4)
            df[input_3]=[() for i in df.index]
            list_check= df[input_6].drop_duplicates().tolist()
            for i,j in itertools.product(list_dict, list_check):
                sorce= f'{j}'
                check=f'{i}'
                filter(condition(df,check,source), df[input_6])
            return df
    if input_1=='n':
        if input_5=='xlsx':
            for file in os.listdir(first_path):
                df= pd.read_excel(first_path+'\\'+file)
                df[input_4]= df[input_4].fillna('نامشخص')
                string_clean(df,input_4)
                df[input_3]=[() for i in df.index]
                list_check= df[input_6].drop_duplicates().tolist()
                for i,j in itertools.product(list_dict, list_check):
                    sorce= f'{j}'
                    check=f'{i}'
                    filter(condition(df,check,source), df[input_6])
                df.to_csv(second_path+'\\'+file.split('.')[0]+'.csv',index=False)
        if input_5=='csv':
            for file in os.listdir(first_path):
                df= pd.read_csv(first_path+'\\'+file)
                df[input_4]= df[input_4].fillna('نامشخص')
                string_clean(df,input_4)
                df[input_3]=[() for i in df.index]
                list_check= df[input_6].drop_duplicates().tolist()
                for i,j in itertools.product(list_dict, list_check):
                    sorce= f'{j}'
                    check=f'{i}'
                    filter(condition(df,check,source), df[input_6])
                df.to_csv(second_path+'\\'+file.split('.')[0]+'.csv',index=False)

Contains

In [ ]:
def translate(df,pd_series,list_dict):
    input_1= input('new column name is: ')
    df[input_1]=None
    for i in list_dict:
        check_value=[i]
        elem_index= df.loc[pd_series.str.contains('|'.join(check_value))].index.to_list()
        try:
            df.loc[elem_index,input_1]=i
        except:
            continue
    return df

Code

In [ ]:
def isic_code (isic_df, path=None, dirr=None, list_dict=None, df=np.nan):
    def insert_code(df, isic):
        check_code= df[input_1].drop_duplicates().tolist()
        check_code= list(filter(None,check_code))
        for i in check_code:
            v= i.replace('(','').replace(')','').replace("'","").split(', ')
            v=[x for x in v if x]
            v=[s.replace(",","") for s in v]
            tuple_i= tuple(v)
            for j in tuple_i:
                for k in isic[input_2]:
                    if j==k:
                        df_select= df.loc[df[input_1]==i]
                        df_index= df_select.index.tolist()
                        j_index= isic.loc[isic[input_2]==k].index.tolist()
                        d= list(isic[input_3][j_index])
                        tuple_code= tuple(k for k in d)
                        if j in df[input_1]:
                            df_index.append(df.index)
                        df.loc[df_index, input_4]= df.loc[df_index, input_4].apply(lambda x: x+tuple_code)
        input_1= input('name of value (tuple) column: ')
        input_2= input('name if isic index column: ')
        input_3= input('name of isic insert value column: ')
        input_4= input('name of new column: ')
        input_5= input(' do you have file?: (n\y)')
        if input_5=='y':
            df[input_4]=[() for i in df.index]
            filter (insert_code(df,isic),df[input_1])
            return df
        if input_5=='n':
            input_6= input('file format is: (xlsx\csv)')
            if input_6=='xlsx':
                for file in os.listdir(path):
                    df= pd.read_excel(path+'\\'+file)
                    df[input_4]=[() for i in df.index]
                    filter(insert_code(df,isic), df[input_1])
                    df.to_csv(dirr+'\\'+file.split('.')[0]+'.csv',index=False)
            if input_6=='csv':
                for file in os.listdir(path):
                    df= pd.read_csv(path+'\\'+file)
                    df[input_4]=[() for i in df.index]
                    filter(insert_code(df,isic), df[input_1])
                    df.to_csv(dirr+'\\'+file.split('.')[0]+'.csv',index=False)

Mode

In [ ]:
def mode_isic(isic,path=None,dirr=None,df=np.nan):
    def mode_filter(df,column_name):
        for i in df[column_name]:
            if len(i)!=0:
                d= list(i)
                v= mode(d)
                h= df.loc[df[input_1]==i].index.tolist()
                df.loc[h,input_2]=v
    input_1= input('name of value (tuple) column: ')
    input_2= input('name of new column: ')
    input_3= input('do you have file: (n\y)')
    if input_3=='y':
        filter(mode_filter(df, input_1), df)
    if input_3=='n':
        input_4= input('file format is: (xlsx\csv)')
        return df
        if input_4=='xlsx':
            for file in os.listdir(path):
                df= pd.read_excel(path+'\\'+file)
                filter(mode_filter(df, input_1),df)
                df.to_csv(dirr+'\\'+file.split('.')[0]+'.csv',index=False)
        if input_4=='csv':
            for file in os.listdir(path):
                df= pd.read_csv(path+'\\'+file)
                filter(mode_filter(df, input_1),df)
                df.to_csv(dirr+'\\'+file.split('.')[0]+'.csv',index=False)

detected detail

In [ ]:
def detected_detail(path,df=None):
    def name_dataframe(path):
        df_name=[]
        for file in os.listdir(path):
            df_name.append(file.split('.')[0])
        df_detected_detail= pd.DataFrame(df_name)
        df_detected_detail.rename(columns={0:'file_name'},inplace=True)
        return df_detected_detail , df_name
    def detected(path, df_list, df=None):
        count_list=[]
        for j in df_list:
            count=0
            df= pd.read_csv(path+'\\'+j+'.csv')
            list_detect= df.isic_name.tolist()
            for i in list_detect:
                v= i.replace('(','').replace(')','').replace("'","").split(', ')
                v=[x for x in v if x]
                v= [s.replace(",","") for s in v]
                tuple_i= tuple(v)
                if not tuple_i:
                    count+=1
                else:
                    count=0
            count_list.append(count)
        return count_list
    df_detected, df_list= name_dataframe(path)
    count_list= detected(path,df_list)
    df_detected['undetect']= count_list
    return df_detected

ISIC

In [ ]:
isic= pd.read_excel(r'Your_own_path')
isic['good_service_cleaned']= isic['good_service'].replace(regex=['-','_'], value=' ')
isic['good_service_cleaned']= isic['good_service_cleaned'].str.strip()
isic['good_service_cleaned']=isic['good_service_cleaned'].replace('  ',' ')
isic['good_service_cleaned']=isic['good_service_cleaned'].str.strip(' ')
isic['good_service_cleaned']=isic['good_service_cleaned'].apply(lambda x: characters.ar_to_fa(x))
isic_good_service= isic['good_service_cleaned']
isic_good_service=isic_good_service.replace(regex= r'\u200c', value=' ')
list_dict= isic_good_service.dropna().drop_duplicates()
list_dict= list_dict.tolist()

In [ ]:
isic['group']=isic['group'].replace(regex=r'\u200c', value=' ')
isic['deste']= isic['daste'].replace(regex=r'\u200c', value=' ')

XLSX to CSV

In [ ]:
excel_to_csv(path=r'your own path',dirr=r'your own path')

Change column name

In [ ]:
change_column_name(df='your own df')

Convert to ISIC name

In [ ]:
regex_translate_tuple_optimize(list_dict, first_path=r'your own path',second_path=r'your own path',df=np.nan)

Code detection

In [ ]:
isic_code(isic, path=r'your own path', dirr=r'your own path',df=np.nan)

Group detection

In [ ]:
isic_code(isic,path=r'your own path',dirr=r'your own path',df=np.nan)

Daste detection

In [ ]:
isic_code(isic, path=r'your own path', dirr=r'your own path', df=np.nan)

Mode detection

In [ ]:
mode_isic(isic, path=r'your own path', dirr=r'your own path', df= np.nan)

detect DF

In [ ]:
df_detcted= detected_detail(path=r'your own path')